In [1]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
from torch_geometric.data import Data

import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

import networkx as nx
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
#Load syndrome matrices and the corresponding eq_class

pickle_in2 = open("MCMC/MCMC-QEC-toric-RL/defect_matrices/s9p1/dict.eq_distr","rb")
pickle_in1 = open("MCMC/MCMC-QEC-toric-RL/defect_matrices/s9p1/dict.defects","rb")




#pickle_in2 = open("MCMC/MCMC-QEC-toric-RL/defect_matrices/s5p05MWPM/few/dict.eq_distr_stdc","rb")
#pickle_in1 = open("MCMC/MCMC-QEC-toric-RL/defect_matrices/s5p05MWPM/few/dict.defects_stdc","rb")

defects = []
eq_distr = []
counter = 0

while 1:
    try:
        eq_distribution = pickle.load(pickle_in2)
        #my_rounded_list = [ round(elem*0.01) for elem in eq_distribution ]
        
        defects.append(pickle.load(pickle_in1))
        eq_distr.append(eq_distribution)

        
        
    except EOFError:
        break


print(len(defects))
print(len(eq_distr))

#print(len(defects1))
#print(len(eq_distr1))


25000
25000


In [3]:
def combine_defect_matrices(qubit_matrix,size):
    #combining the vertex- and plaquette defect matrices
    a = np.array(qubit_matrix[0])
    b = np.array(qubit_matrix[1])
    #print(a)
    #print(b)
    
    a_reshaped = np.zeros((size,size))
    b_reshaped = np.zeros((size,size))

    a_reshaped[:a.shape[0],:a.shape[1]] = a
    b_reshaped[:b.shape[0],:b.shape[1]] = b

    for ix in range(size-1):
        if ix ==0:
            a_reshaped = np.insert(a_reshaped,ix+1,0,axis=1)
            a_reshaped = np.insert(a_reshaped,ix,0,axis=0)
        
            b_reshaped = np.insert(b_reshaped,ix,0,axis=1)
            b_reshaped = np.insert(b_reshaped,ix+1,0,axis=0)
        else:
            a_reshaped = np.insert(a_reshaped,(ix*2)+1,0,axis=1)
            a_reshaped = np.insert(a_reshaped,ix*2,0,axis=0)
        
            b_reshaped = np.insert(b_reshaped,ix*2,0,axis=1)
            b_reshaped = np.insert(b_reshaped,(ix*2)+1,0,axis=0)
        
    combine = a_reshaped+b_reshaped
    #print(combine) 
    return combine

def manhattan_dist(positions,nbr_of_defects):
    manhattan_distances = np.zeros((nbr_of_defects,nbr_of_defects))
    for ix in range(nbr_of_defects):
        for jx in range(nbr_of_defects):
            if ix == jx:manhattan_distances[ix][jx] = 0
            else:
                manhattan_distances[ix][jx] = 1/(abs(positions[ix,0]-positions[jx,0])+abs(positions[ix,1]-positions[jx,1]))

    return manhattan_distances

def distance_to_boundary(combined_matrix,idx,size):
    
    #size = 5
    defect_positions = np.argwhere(combined_matrix)
    nbr_of_defects=len(defect_positions)
    new_pos= np.zeros((nbr_of_defects,2))
    
    for ix in range(nbr_of_defects):
        if defect_positions[ix][0]%2 == 0 :
            aa = defect_positions[ix,1]*0.5
            new_pos[ix] = [defect_positions[ix,0]*0.5,aa]
        else:
            bb = defect_positions[ix,0]*0.5
            new_pos[ix] = [bb,defect_positions[ix,1]*0.5]

    ##################
    #print(new_pos)
    manhattan_distances = manhattan_dist(new_pos,nbr_of_defects)
    #print(manhattan_distances)
    
    #from adj.matrix to edgelist
    graph = nx.from_numpy_matrix(manhattan_distances)
    #edges to be directed both ways
    graph = nx.to_directed(graph)
    layout= nx.spring_layout(graph)
    
    ####plot one graph
    #if idx == len(defects)-1:
        #f = plt.figure()
        #nx.draw(graph,ax=f.add_subplot(111),with_labels=True,arrowsize=2, arrowstyle='fancy')
        #f.savefig("defect_matrices/Graph.png", format="PNG")
    ####

    file_2 = open('data/stdc/d9p1/edgelist.txt','ab')        
    nx.write_edgelist(graph, file_2)
    
    #new_line = '\n'
    #file_2.write(new_line.encode('utf-8'))

    file_3 = open('data/stdc/d9p1/graph_info.txt','a')        
    file_3.write(str(nbr_of_defects)+'\n')
    
    #file_2.write(new_line.encode('utf-8'))

    
    ##################
    
    order_list_vertex = []
    order_list_plaquette = []
    for ix in range(nbr_of_defects):
        if (round(new_pos[ix,0],2)-int(new_pos[ix,0]) != 0.):
            #print(new_pos[ix,0])
            order_list_vertex.append(ix)
        if (round(new_pos[ix,1],2)-int(new_pos[ix,1]) != 0.):
            #print(new_pos[ix,1])
            order_list_plaquette.append(ix)

    order_array_plaquette = np.array(order_list_plaquette)
    order_array_vertex = np.array(order_list_vertex)

    #print(len(order_array_plaquette))
    #print(len(order_array_vertex))
    #print(order_array_vertex)
    #print(order_array_plaquette)

    #print(new_pos[order_array_vertex])
    #print(new_pos[order_array_plaquette])
    
    ##########
    #special case when there are only vertex defects
    if len(order_array_plaquette) == 0:
        distance_vertex_boundary = np.zeros(len(order_array_vertex))
        for ix in range(len(order_array_vertex)):
            dist = new_pos[order_array_vertex[ix]]
            if dist[0] >= size*0.5:
                dist[0] = -(size-dist[0])
    
            distance_vertex_boundary[ix] = dist[0]
    
    
    
        boundary_vertex = np.around(distance_vertex_boundary+0.1)*0.5
    
        boundary_v = np.ones((len(boundary_vertex),2))
        boundary_v = -1*boundary_v
        boundary_v[:,0] = boundary_vertex

        node_features = np.zeros((nbr_of_defects,2))
        node_features[order_array_vertex] = boundary_v
    
    
    ############
    #special case when there are only plaq. defects
    elif len(order_array_vertex) == 0:
        distance_plaquette_boundary = np.zeros(len(order_array_plaquette))

        for ix in range(len(order_array_plaquette)):
            dist = new_pos[order_array_plaquette[ix]]
            if dist[1] >= size*0.5:
                dist[1] = -(size-dist[1])
    
            distance_plaquette_boundary[ix] = dist[1]
    

        boundary_plaquette = np.around(distance_plaquette_boundary+0.1)*0.5

    
        boundary_p = np.ones((len(boundary_plaquette),2))
        boundary_p[:,0] = boundary_plaquette
        
        
        node_features = np.zeros((nbr_of_defects,2))
        node_features[order_array_plaquette] = boundary_p
        
        #print(node_features)
    
        
    ##########
    #typical case
    else:
        distance_vertex_boundary = np.zeros(len(order_array_vertex))
        distance_plaquette_boundary = np.zeros(len(order_array_plaquette))

        for ix in range(len(order_array_vertex)):
            dist = new_pos[order_array_vertex[ix]]
            if dist[0] >= size*0.5:#2.5:
                dist[0] = -(size-dist[0])
    
            distance_vertex_boundary[ix] = dist[0]
    
        for ix in range(len(order_array_plaquette)):
            dist = new_pos[order_array_plaquette[ix]]
            if dist[1] >= size*0.5:#2.5:
                dist[1] = -(size-dist[1])
    
            distance_plaquette_boundary[ix] = dist[1]
    

        boundary_vertex = np.around(distance_vertex_boundary+0.1)*0.5
        boundary_plaquette = np.around(distance_plaquette_boundary+0.1)*0.5

        
        boundary_v = np.ones((len(boundary_vertex),2))
        boundary_v = -1*boundary_v
        boundary_v[:,0] = boundary_vertex

        boundary_p = np.ones((len(boundary_plaquette),2))
        boundary_p[:,0] = boundary_plaquette
    
        node_features = np.zeros((nbr_of_defects,2))
        node_features[order_array_vertex] = boundary_v
        node_features[order_array_plaquette] = boundary_p
    
    return node_features 


#caveman function to check balance of classes
def conditions(eq_distr,eq_class):
    if np.round(eq_distr[0]*0.01) == 1:
        eq_class[0] = eq_class[0] + 1
    
    elif np.round(eq_distr[1]*0.01) == 1:
        eq_class[1] = eq_class[1] + 1
    
    elif np.round(eq_distr[2]*0.01) == 1:
        eq_class[2] = eq_class[2] + 1
    
    else: eq_class[3] = eq_class[3] + 1
        
    return eq_class


    
    

In [4]:
file_edge=open('data/stdc/d9p1/node_features.txt','a')
pickle_eq_dist = open("data/stdc/d9p1/dict.eq_distr","ab")
pickle_defect_matrices = open("data/stdc/d9p1/dict.defect","ab")

eq_class_counter = np.zeros(4)

zero_def = 0
single_def = 0

defect_test = []
eq_dist_test = []
_, size = np.array(defects[0][0]).shape
for ix in range(len(defects)):
    if (len(np.argwhere(defects[ix][0]))+len(np.argwhere(defects[ix][1]))) == 0:
        zero_def +=1
        continue
    
    
    
    #### REMOVE FOR REGULAR RUN, ONLY FOR FEW DEFECT SIM.
    #if (len(np.argwhere(defects[ix][0]))+len(np.argwhere(defects[ix][1]))) > 8 :
    #    continue
    
    if (len(np.argwhere(defects[ix][0]))+len(np.argwhere(defects[ix][1]))) == 1:    
        single_def +=1
        continue
    ##################################################
    
    #the number of syndromes wanted of each class, eg 3500
    if np.round(eq_distr[ix][0]*0.01) == 1 and eq_class_counter[0] >= 3500:
        continue
    if np.round(eq_distr[ix][1]*0.01) == 1 and eq_class_counter[1] >= 3500:
        continue
    if np.round(eq_distr[ix][2]*0.01) == 1 and eq_class_counter[2] >= 3500:
        continue
    if np.round(eq_distr[ix][3]*0.01) == 1 and eq_class_counter[3] >= 3500:
        continue
    
    
    eq_class_counter = conditions(eq_distr[ix],eq_class_counter)
    
    
    
    eq_dist_test.append(eq_distr[ix])
    pickle.dump(eq_distr[ix],pickle_eq_dist)
    pickle.dump(defects[ix],pickle_defect_matrices)
    
    combined_matrix = combine_defect_matrices(defects[ix],size)
    node_features = distance_to_boundary(combined_matrix,ix,size)
    #print(node_features)
    
    np.savetxt(file_edge,node_features,fmt='%.1f')
    file_edge.write("\n")

pickle_eq_dist.close()
pickle_defect_matrices.close()
file_edge.close()

In [5]:
eq_class_counter

array([3500., 3500., 3500., 3436.])